In [1]:
import os, time
import regex as re
import pandas as pd
from dotenv import load_dotenv
from map_industries import make_ind_dict, make_tag_dict
from write_properties import create_company_property
from write_companies import update_company, batch_update_company
from industry_grouping import aggregate_industries, map_tags_to_investors, make_industry_options

load_dotenv()

DBDIR = "C:/Users/galon/Sputnik ATX Team Dropbox/Programming Datasets"
RAW_DIR = os.path.join(DBDIR, "data", "raw")
CLEAN_DIR = os.path.join(DBDIR, "data", "clean")
IND_DIR = os.path.join(CLEAN_DIR, "industry_mapping")
VC_INDUSTRY_COLS = os.path.join(RAW_DIR, "hs", "vc_industry_columns.csv")
STARTPATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase', 'cb_starts_main_scraped.csv')
INVPATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase', 'cb_vc_main_merged.csv')
MAPPING_OUTPATH = os.path.join(IND_DIR, "cb_starts_mapped.csv")
START_HERF_PATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase','cb_inv_overview_scraped.csv')
VC_PATH = os.path.join(RAW_DIR, 'hs', 'vc_list_export.csv')

### Column inpaths

[{'label': 'DeepTech',
  'value': 'DeepTech',
  'displayOrder': 0,
  'hidden': False},
 {'label': 'Boating and Marine',
  'value': 'Boating_and_Marine',
  'displayOrder': 1,
  'hidden': False},
 {'label': 'Education',
  'value': 'Education',
  'displayOrder': 2,
  'hidden': False},
 {'label': 'Health', 'value': 'Health', 'displayOrder': 3, 'hidden': False},
 {'label': 'Accounting',
  'value': 'Accounting',
  'displayOrder': 4,
  'hidden': False},
 {'label': "Men's Products",
  'value': "Men's_Products",
  'displayOrder': 5,
  'hidden': False},
 {'label': 'Design', 'value': 'Design', 'displayOrder': 6, 'hidden': False},
 {'label': 'Legal', 'value': 'Legal', 'displayOrder': 7, 'hidden': False},
 {'label': 'Search Software',
  'value': 'Search_Software',
  'displayOrder': 8,
  'hidden': False},
 {'label': 'Software',
  'value': 'Software',
  'displayOrder': 9,
  'hidden': False},
 {'label': 'Pet Care',
  'value': 'Pet_Care',
  'displayOrder': 10,
  'hidden': False},
 {'label': 'Crowdsourc